<a href="https://colab.research.google.com/github/sumanyurosha/tensorflow-specialization/blob/master/Practice/Practicing_Text_Generation_one_more_time.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [70]:
import tensorflow as tf
import numpy as np
import os
from tensorflow import keras

# **Downloading the Dataset**

In [71]:
shakespeare_url = "https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt"
filepath = keras.utils.get_file("shakespeare", shakespeare_url)

with open(filepath, "r") as f:
    text_data = f.read()

print("The size of Dataset is :{}".format(len(text_data)))

The size of Dataset is :1115394


# **Analyzing the Dataset**

In [72]:
print(text_data[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [73]:
vocab = sorted(set(text_data))
print("Unique characters in the Dataset are : {}".format(len(vocab)))

Unique characters in the Dataset are : 65


In [74]:
for char, i in zip(vocab, range(20)):
    print("{} : {}".format(i+1, repr(char)))

1 : '\n'
2 : ' '
3 : '!'
4 : '$'
5 : '&'
6 : "'"
7 : ','
8 : '-'
9 : '.'
10 : '3'
11 : ':'
12 : ';'
13 : '?'
14 : 'A'
15 : 'B'
16 : 'C'
17 : 'D'
18 : 'E'
19 : 'F'
20 : 'G'


# **Creating a Vocabulary out of the Dataset**

In [75]:
char2idx = {char:i for i, char in enumerate(vocab)}
idx2char = np.array(vocab)

# converting text into a sequence of integers
text_as_int = np.array([char2idx[char] for char in text_data])

print(repr(text_data[:13]))
print(text_as_int[:13])

'First Citizen'
[18 47 56 57 58  1 15 47 58 47 64 43 52]


# **Setting Hyperparameters**

In [76]:
vocab_size = len(vocab)
batch_size = 32
seq_len = 100
embedding_dim = 256
rnn_units = 1024

# **Creating a Dataset for our Model**

In [77]:
# creating a Dataset with batches equal to sequence length + 1 (+ 1 for target)

dataset = tf.data.Dataset.from_tensor_slices(text_as_int)
dataset = dataset.batch(seq_len + 1, drop_remainder=True)

# shape should be (101, )
dataset

<BatchDataset shapes: (101,), types: tf.int64>

# **Creating Input and Target Values for our Model**

In [78]:
def seperate_input_target(chunk):
    # all except the last
    input_text = chunk[:-1]
    # all except the first
    output_text = chunk[1:]

    return input_text, output_text


In [79]:
dataset = dataset.map(seperate_input_target)

# shape should be (100, ), (100, )
dataset

<MapDataset shapes: ((100,), (100,)), types: (tf.int64, tf.int64)>

# **Checking what our model will see**

In [80]:
for input, output in dataset.take(1):
    print("Input : {}".format(repr("".join([idx2char[char] for char in input]))))
    print()
    print("Output: {}".format(repr("".join([idx2char[char] for char in output]))))

Input : 'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'

Output: 'irst Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '


# **Creating batches of our Dataset**

In [81]:
dataset = dataset.shuffle(10000).batch(batch_size, drop_remainder=True)

# shape should be (32, 100), (32, 100)
dataset

<BatchDataset shapes: ((32, 100), (32, 100)), types: (tf.int64, tf.int64)>

# **Creating a Model for our Dataset**

In [82]:
def build_model(batch_size, rnn_units, vocab_size, embedding_dim):

    model = keras.models.Sequential([
        keras.layers.Embedding(vocab_size, embedding_dim, 
                               batch_input_shape=[batch_size,None]),
        keras.layers.GRU(128, dropout=0.2, stateful=True, return_sequences=True),
        keras.layers.GRU(128, dropout=0.2, stateful=True, return_sequences=True),
        keras.layers.TimeDistributed(keras.layers.Dense(vocab_size,
         activation="softmax"))
    ])

    return model

In [83]:
def loss(labels, logits):
    return keras.losses.sparse_categorical_crossentropy( labels, logits,
                                                        from_logits=True,)

In [84]:
model = build_model(batch_size=batch_size, 
                    rnn_units=rnn_units,
                    vocab_size=vocab_size,
                    embedding_dim=embedding_dim)

model.summary()

model.compile(loss="sparse_categorical_crossentropy",
              optimizer="adam")

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_8 (Embedding)      (32, None, 256)           16640     
_________________________________________________________________
gru_8 (GRU)                  (32, None, 128)           148224    
_________________________________________________________________
gru_9 (GRU)                  (32, None, 128)           99072     
_________________________________________________________________
time_distributed_7 (TimeDist (None, None, 65)          8385      
Total params: 272,321
Trainable params: 272,321
Non-trainable params: 0
_________________________________________________________________


# **Checking the Model beahviour and Output shape**

In [85]:
for input, output in dataset.take(1):  
    predicted_example = model(input)
    print(predicted_example.shape)

(32, 100, 65)


In [86]:
predicted_example[0][0]

<tf.Tensor: shape=(65,), dtype=float32, numpy=
array([0.01535656, 0.01547176, 0.01537743, 0.01522947, 0.01519165,
       0.01546737, 0.01548958, 0.01542859, 0.01532648, 0.01538258,
       0.01534014, 0.01533251, 0.01539102, 0.01533142, 0.01520745,
       0.01551652, 0.0154529 , 0.01543395, 0.01548969, 0.01544755,
       0.01539544, 0.01545648, 0.01540778, 0.01536581, 0.01552243,
       0.01527205, 0.01532785, 0.01541418, 0.01546018, 0.01531322,
       0.01538095, 0.01531458, 0.01531058, 0.01557555, 0.01537086,
       0.01536181, 0.01537607, 0.01539904, 0.01542821, 0.01540883,
       0.01540503, 0.01524932, 0.01538644, 0.01539191, 0.01533041,
       0.01555246, 0.01540336, 0.01532918, 0.01537277, 0.0153609 ,
       0.01554938, 0.01541056, 0.01542772, 0.01538872, 0.01543457,
       0.01547509, 0.0152868 , 0.01527127, 0.0153402 , 0.01533103,
       0.01529409, 0.01530994, 0.01555583, 0.01530431, 0.01531223],
      dtype=float32)>

In [87]:
# select a sequence from the batch
# here predicted_example[0] has a shape (seq_len, vocab_size)
# where row[i, :] contains prob for each word in the vocab for the ith word in the sequence
prediction = tf.random.categorical(predicted_example[0], num_samples=1)
prediction = np.argmax(predicted_example[0], axis=1)
#prediction = tf.squeeze(prediction, axis=-1)
prediction

array([33, 18, 18, 18, 18, 24, 24, 24, 24, 24, 24, 36, 37, 37, 37, 37, 55,
       55, 55, 47, 36, 64, 28, 18, 21, 18, 18, 61, 61, 62, 62, 31, 64, 61,
       61, 64, 24, 24, 24, 24,  1, 26, 52, 18, 64, 64, 21, 13, 18, 18, 21,
       64, 18, 21, 21, 30, 30, 33, 18, 18, 22, 64, 13, 18, 61, 61, 18,  2,
        2,  2, 53, 37, 23, 24, 24, 24, 24, 24, 43, 61, 18, 18, 21, 19, 18,
       18, 60, 18, 18, 24, 24, 24, 37, 37, 62, 30, 62, 18, 24, 31])

In [88]:
prediction.shape

(100,)

In [89]:
# we should expect very random sequence because the model is not trained yet
print(repr("".join([idx2char[c] for c in prediction])))

'UFFFFLLLLLLXYYYYqqqiXzPFIFFwwxxSzwwzLLLL NnFzzIAFFIzFIIRRUFFJzAFwwF!!!oYKLLLLLewFFIGFFvFFLLLYYxRxFLS'


# **Creating a Checkpoint Callback for saving our Model**

In [90]:
checkpoint_dir = "/checkpoints"

checkpoint_prefix = os.path.join(checkpoint_dir, "{epoch}")

checkpoint_callback = keras.callbacks.ModelCheckpoint(checkpoint_prefix,
                                                      save_weights_only=True)

# **Training our Model**

In [91]:
history = model.fit(dataset, epochs=30, callbacks=[checkpoint_callback], verbose=1)

Epoch 1/30
345/345 [==============================] - 4s 12ms/step - loss: 2.5074
Epoch 2/30
345/345 [==============================] - 4s 12ms/step - loss: 1.9505
Epoch 3/30
345/345 [==============================] - 4s 12ms/step - loss: 1.7941
Epoch 4/30
345/345 [==============================] - 4s 12ms/step - loss: 1.7103
Epoch 5/30
345/345 [==============================] - 4s 12ms/step - loss: 1.6590
Epoch 6/30
345/345 [==============================] - 4s 12ms/step - loss: 1.6243
Epoch 7/30
345/345 [==============================] - 4s 12ms/step - loss: 1.5996
Epoch 8/30
345/345 [==============================] - 4s 12ms/step - loss: 1.5776
Epoch 9/30
345/345 [==============================] - 4s 12ms/step - loss: 1.5625
Epoch 10/30
345/345 [==============================] - 4s 12ms/step - loss: 1.5490
Epoch 11/30
345/345 [==============================] - 4s 12ms/step - loss: 1.5379
Epoch 12/30
345/345 [==============================] - 4s 12ms/step - loss: 1.5272
Epoch 13/30
3

# **Building a new model with saved weights but different batch size for Text Generation**

In [92]:
tf.train.latest_checkpoint(checkpoint_dir)

'/checkpoints/30'

In [93]:
model = build_model(batch_size=1, vocab_size=vocab_size,
                    embedding_dim=embedding_dim, rnn_units=rnn_units)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

# here the batch_size should be 1
model.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_9 (Embedding)      (1, None, 256)            16640     
_________________________________________________________________
gru_10 (GRU)                 (1, None, 128)            148224    
_________________________________________________________________
gru_11 (GRU)                 (1, None, 128)            99072     
_________________________________________________________________
time_distributed_8 (TimeDist (None, None, 65)          8385      
Total params: 272,321
Trainable params: 272,321
Non-trainable params: 0
_________________________________________________________________


In [94]:
def generate_text(model, start_string):

    text_length = 1000
    text_generated = []

    input_sequence = np.array([char2idx[c] for c in start_string])
    input_sequence = tf.expand_dims(input_sequence, 0)

    model.reset_states()
    for i in range(text_length):
        prediction = model(input_sequence)
        prediction = tf.squeeze(prediction, 0)
        prediction = tf.math.log(prediction)
        prediction_id = tf.random.categorical(prediction, num_samples=1)[-1, 0].numpy()

        input_sequence = tf.expand_dims([prediction_id], 0)

        text_generated.append(idx2char[prediction_id])
        

    return start_string + "".join(text_generated)


In [95]:
print(generate_text(model, "Romeo:"))

Romeo:
In heaven death, as in the buny 'eg, my lords,
The first butney, let me this glory on my not:
List and ruilth sad day:
All Henry that I have indeed this your peing?

Lord:
Petake us never scorn;
And still well, such Lancaster, methink bean all widowr.

KING EDWARD IV:
Say, nor haughs and him.

POMPEY:
Ey, nor much, I'll had fales:
Two woman is so Of segners.

FLORIZEL:
Sweet, good quarred their son: I were immether.

KING EDWARD IV:
Come, that were as doom,---

KING HERRY VI:
Sir? A wild chambors of the malsh!
His loss te in the wedd-missed to the sire
Than eye a Montague;
I please--Boldal, if once was need becauson'-thought,
And you or though leave a licken than no grace, what of my hasom: so fellow?

MERCUTIO:
Imaballal life adverting words of the soulf his court:
He will make it but to your talk you, day!

THOMAMERLE:
When I beseech you are a reasons; I think not on an't so as you sworn: in the head, what courvers
Is be
I never leands all under-boy Down:
I would not ralner wi